In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from wrf import interplevel, getvar, destagger
CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
dx = 32000.
dy = 32000.
nx = 250
ny = 225
nz_coamps = 100
nz_wrf = 100
shape_coamps = [nz_coamps, ny, nx]
shape_wrf = [nz_wrf, ny, nx]

wrf_dir = '/p/work1/lloveras/adj_32km/in_files/wrfin_adj_100'
coamps_dir = '/p/work1/lloveras/adj_32km/input/files_from_jim/adj_48/'

#wave = False

In [ ]:
ncfile = Dataset(wrf_dir,'r+')
wrf_u = getvar(ncfile,'U')
wrf_v = getvar(ncfile,'V')
wrf_th = getvar(ncfile,'T')
wrf_p = getvar(ncfile,'P')
wrf_qv = getvar(ncfile,'QVAPOR')
wrf_full_p = getvar(ncfile,'pressure')


coamps_u = np.fromfile(coamps_dir + 'aaauu1_sig_020007_000050_1a0250x0225_2022070500_00000000_fcstfld', dtype='>f4')
coamps_v = np.fromfile(coamps_dir + 'aaavv1_sig_020007_000050_1a0250x0225_2022070500_00000000_fcstfld', dtype='>f4')
coamps_ex = np.fromfile(coamps_dir + 'aaapp1_sig_020007_000050_1a0250x0225_2022070500_00000000_fcstfld', dtype='>f4')
coamps_th = np.fromfile(coamps_dir + 'aaath1_sig_020007_000050_1a0250x0225_2022070500_00000000_fcstfld', dtype='>f4')
coamps_qv = np.fromfile(coamps_dir + 'aaaqv1_sig_020007_000050_1a0250x0225_2022070500_00000000_fcstfld', dtype='>f4')
coamps_full_p = np.fromfile(coamps_dir + 'ttlprs_sig_020007_000050_1a0250x0225_2022070500_00000000_fcstfld', dtype='>f4')


In [ ]:
coamps_u = np.flip(np.reshape(coamps_u, shape_coamps),axis=0)
coamps_v = np.flip(np.reshape(coamps_v, shape_coamps),axis=0)
coamps_ex = np.flip(np.reshape(coamps_ex, shape_coamps),axis=0)
coamps_th = np.flip(np.reshape(coamps_th, shape_coamps),axis=0)
coamps_qv = np.flip(np.reshape(coamps_qv, shape_coamps),axis=0)
coamps_full_p = np.flip(np.reshape(coamps_full_p, shape_coamps),axis=0)

coamps_pert_ex = (coamps_full_p/P0)**(RD/CP) + coamps_ex
coamps_pert_p = P0*coamps_pert_ex**(CP/RD)
coamps_p = (coamps_pert_p - coamps_full_p)*100.


In [ ]:
umax = np.amax(np.abs(coamps_u))
vmax = np.amax(np.abs(coamps_v))
thmax = np.amax(np.abs(coamps_th))
pmax = np.amax(np.abs(coamps_p))
qvmax = np.amax(np.abs(coamps_qv))
print(umax, vmax, thmax, pmax, qvmax*1000)


In [ ]:
u_pert = np.zeros([nz_wrf,ny,nx+1])
v_pert = np.zeros([nz_wrf,ny+1,nx])
p_pert = np.zeros(shape_wrf)
th_pert = np.zeros(shape_wrf)
qv_pert = np.zeros(shape_wrf)
for k in range(nz_wrf):
    u_pert[k,:,:-1] = interplevel(coamps_u, coamps_full_p, wrf_full_p[k,:,:])
    v_pert[k,:-1,:] = interplevel(coamps_v, coamps_full_p, wrf_full_p[k,:,:])
    th_pert[k,:,:] = interplevel(coamps_th, coamps_full_p, wrf_full_p[k,:,:])
    p_pert[k,:,:] = interplevel(coamps_p, coamps_full_p, wrf_full_p[k,:,:])
    qv_pert[k,:,:] = interplevel(coamps_qv, coamps_full_p, wrf_full_p[k,:,:])
    
u_pert[:,:,-1] = u_pert[:,:,0]
v_pert[:,-1,:] = v_pert[:,-2,:]

u_pert = np.nan_to_num(u_pert)
v_pert = np.nan_to_num(v_pert)
th_pert = np.nan_to_num(th_pert)
p_pert = np.nan_to_num(p_pert)
qv_pert = np.nan_to_num(qv_pert)


In [ ]:
#l = 1000. #210.
#lm = l*1000.

#u_pert_wave = np.zeros([nz_wrf,ny,nx+1])
#v_pert_wave = np.zeros([nz_wrf,ny+1,nx])
#p_pert_wave = np.zeros(shape_wrf)
#th_pert_wave = np.zeros(shape_wrf)
#qv_pert_wave = np.zeros(shape_wrf)

#for i in range(nx):
#    for j in range(ny):
#        u_pert_wave[:,j,i] = 2*u_pert[:,j,i]*np.sin(2*np.pi*i*dx/lm)*np.sin(2*np.pi*j*dx/lm)
#        v_pert_wave[:,j,i] = 2*v_pert[:,j,i]*np.sin(2*np.pi*i*dx/lm)*np.sin(2*np.pi*j*dx/lm)
#        p_pert_wave[:,j,i] = 2*p_pert[:,j,i]*np.sin(2*np.pi*i*dx/lm)*np.sin(2*np.pi*j*dx/lm)
#        th_pert_wave[:,j,i] = 2*th_pert[:,j,i]*np.sin(2*np.pi*i*dx/lm)*np.sin(2*np.pi*j*dx/lm)
#        qv_pert_wave[:,j,i] = 2*qv_pert[:,j,i]*np.sin(2*np.pi*i*dx/lm)*np.sin(2*np.pi*j*dx/lm)

#if wave:
#    u_pert = u_pert_wave
#    v_pert = v_pert_wave
#    p_pert = p_pert_wave
#    th_pert = th_pert_wave
#    qv_pert = qv_pert_wave


In [ ]:
#div = 1.

ncfile.variables['U'][0,:,:,:] = wrf_u + u_pert/100.
ncfile.variables['V'][0,:,:,:] = wrf_v + v_pert/100.
ncfile.variables['T'][0,:,:,:] = wrf_th + th_pert/100.
ncfile.variables['P'][0,:,:,:] = wrf_p + p_pert/100.
ncfile.variables['QVAPOR'][0,:,:,:] = wrf_qv + qv_pert/100.

#ncfile.variables['U_2'][0,:,:,:] = wrf_u + u_pert*div
#ncfile.variables['V_2'][0,:,:,:] = wrf_v + v_pert*div
#ncfile.variables['T_2'][0,:,:,:] = wrf_th + th_pert*div


In [ ]:
ncfile.close()
